In [1]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud
from sklearn.model_selection import cross_val_score

import seaborn as sns
import matplotlib.pyplot as plt
import re
import pandas as pd
import joblib
import numpy as np
import xgboost as xgb


In [2]:
dataset_dir = 'sentiment140'
# dataset_dir = 'imdb'
# dataset_dir = 'coronaNLP'

n_gram = (1, 2)

# importing the processed dataframe
df = joblib.load(f'../dataframes/df_{dataset_dir}.pkl')

df.head()


,tweet,sentiment
0,awww bummer shoulda got david carr third day,0
1,upset cant updat facebook text might cri resul...,0
2,dive mani time ball manag save 50 rest go bound,0
3,whole bodi feel itchi like fire,0
4,behav im mad whi becaus cant see,0


In [3]:

X = df.iloc[:, 0]


y = df.iloc[:, 1]

X, y


(0               awww bummer shoulda got david carr third day
 1          upset cant updat facebook text might cri resul...
 2            dive mani time ball manag save 50 rest go bound
 3                            whole bodi feel itchi like fire
 4                           behav im mad whi becaus cant see
                                  ...                        
 1599995                           woke school best feel ever
 1599996      thewdbcom veri cool hear old walt interview â«
 1599997                         readi mojo makeov ask detail
 1599998    happi 38th birthday boo alll time tupac amaru ...
 1599999                                 happi charitytuesday
 Name: tweet, Length: 1583691, dtype: object,
 0          0
 1          0
 2          0
 3          0
 4          0
           ..
 1599995    1
 1599996    1
 1599997    1
 1599998    1
 1599999    1
 Name: sentiment, Length: 1583691, dtype: int32)

In [4]:

tfidf = joblib.load(
    f"../vectors/vectorizer_{dataset_dir}_{n_gram}.pkl")
tfidf


TfidfVectorizer(max_features=402749, ngram_range=(1, 2), stop_words='english',
                tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)

In [5]:

X = tfidf.transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)

X_train.shape, y_train.shape


((1108583, 402749), (1108583,))

In [6]:
estimators = []
estimators.append(('MNB', MultinomialNB()))
estimators.append(('CNB', ComplementNB()))
# estimators.append(('XGB',
#                   xgb.XGBClassifier(max_depth=50, use_label_encoder=False)))
estimators.append(('LSVC', LinearSVC()))
estimators.append(('LRG', LogisticRegression()))

models = [
    MultinomialNB(),
    ComplementNB(),
    # xgb.XGBClassifier(max_depth=50, use_label_encoder=False),
    LinearSVC(),
    LogisticRegression(),
    VotingClassifier(estimators=estimators, voting='hard'),
]

model_to_use = 0

model_idx = model_to_use

# model_name = str(models[model_idx]).split('(')[0]
# model_name += 'S'

In [18]:
# for i, model in enumerate(models):
#     model_name = str(models[i])[:-2]
#     print(f"-----Training {model_name}-----")
#     clf = model
#     clf.fit(X_train, y_train)
#     cross_clf = cross_val_score(clf, X, y)
#     print("Cross Validation score = ", cross_clf)
#     train_acc_clf = clf.score(X_train, y_train)
#     test_acc_clf = clf.score(X_test, y_test)
#     print("Train accuracy ={:.2f}%".format(train_acc_clf*100))
#     print("Test accuracy ={:.2f}%".format(test_acc_clf*100))
#     y_pred = clf.predict(X_test)
#     print(classification_report(y_test, y_pred))
#     acc = int(accuracy_score(y_test, y_pred)*100)
#     print("ACC: ", acc)
#     joblib.dump(clf,
#                 f'../models/{model_name}_{dataset_dir}_{acc}.pkl')
    

In [19]:
# for i, model in enumerate(models):
#     model_name = str(model).split('(')[0]
#     clf = model
#     clf = clf.fit(X_train, y_train)
#     # y_pred = clf.predict(X_test)
#     train_acc_clf = clf.score(X_train, y_train)
#     print(train_acc_clf)
#     # print(classification_report(y_test, y_pred))
#     # acc = int(accuracy_score(y_test, y_pred)*100)
#     # joblib.dump(clf,
#     #             f'../models/{model_name}_{dataset_dir}_{acc}_{n_gram}.pkl')


0.8216615264711798
0.8208027725483793
0.8801731579863664


C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.8260779752170113


C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.8331139842483603


In [20]:
params = [
    {
        'fit_prior': (False, True),
        'alpha': (1, 0.1, 0.01, 0.001)
    }, 
    {
        'fit_prior': (False, True),
        'norm': (False, True),
        'alpha': (1, 0.1, 0.01, 0.001)
    },
    {
        'booster': ('gbtree', 'gblinear', 'dart'),
        'eta': (0.1, 0, 25, 0.4, 0.5), 
    },
    {
        'C': ('1', '0.5', '0.25'),
        'kernel': ('rfb', 'linear', 'poly', 'sigmoid'),
    },
    {
        'penalty': ('l2', 'none'),
        'C': np.logspace(-4, 4, 10),
        'solver': ('sag', 'saga', 'newton-cg'),
        'max_iter': (100, 1000, 2500, 5000)
    }
]

In [7]:
clf = models[model_idx]

# parameters = params[model_idx]

# clf = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=5, verbose=True) if model_idx != -1 else clf
# clf


In [8]:

clf = clf.fit(X_train, y_train)
# clf.best_estimator_


In [ ]:

# fig, ax = plt.subplots(figsize=(30, 30))
# xgb.plot_tree(clf, num_trees=4, ax=ax)
# plt.show()


In [9]:

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.77      0.76      0.77    237056
           1       0.77      0.78      0.77    238052

    accuracy                           0.77    475108
   macro avg       0.77      0.77      0.77    475108
weighted avg       0.77      0.77      0.77    475108



In [10]:
prediction_probs = clf.predict_proba(X_test)

In [11]:
prediction_probs

array([[0.12190435, 0.87809565],
       [0.18118281, 0.81881719],
       [0.28937892, 0.71062108],
       ...,
       [0.57378174, 0.42621826],
       [0.45363124, 0.54636876],
       [0.11549447, 0.88450553]])

In [ ]:

# print("Best: %f using %s" % (clf.best_score_,
#                              clf.best_params_))
# means = clf.cv_results_['mean_test_score']
# stds = clf.cv_results_['std_test_score']
# params = clf.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:

acc = int(accuracy_score(y_test, y_pred)*100)
acc

In [ ]:
# test_tweet = "groceri store"
# vector = tfidf.transform([test_tweet])

# print(clf.predict(vector))


In [ ]:

# exporting the pipeline
joblib.dump(clf,
            f'../models/mnb_{dataset_dir}_{acc}_{n_gram}.pkl')
